In [16]:
%load_ext autoreload
%autoreload
from DBConnectionUtils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def getIMDBConn():
    conn = psycopg2.connect(dbname="IMDB", host='/tmp', user="kdeeds", port='5555')
    conn.autocommit = True
    conn.cursor().execute("Load 'pg_hint_plan';")
    dbConn = DatabaseConnection(conn)
    return dbConn

def getSizeEstimateAndActual(query):
    dbConn = getIMDBConn()
    est,size = dbConn.getSizeEstimateAndActual(query)
    dbConn.close()
    return est, size

def getFreqBound(query, dbStats, VA):
    return dbStats.frequencyBound(query, VA)

In [8]:
tableNamesList = [ "cast_info", "aka_name", "aka_title",
             "char_name", "comp_cast_type", "company_name",
             "company_type", "complete_cast", "info_type",
             "keyword", "kind_type", "link_type",
             "movie_companies", "movie_info_idx", "movie_info",
             "movie_keyword", "movie_link", "name",
             "person_info", "role_type", "title"]

tableJoinColsList = [
                   ["id", "person_id", "movie_id", "person_role_id", "role_id"],
                   ["id", "person_id"],
                   ["id", "movie_id", "kind_id"],
                   ["id"],
                   ["id", "kind"],
                   ["id"],
                   ["id", "kind"],
                   ["id", "movie_id", "subject_id", "status_id"],
                   ["id", "info"],
                   ["id"],
                   ["id", "kind"],
                   ["id", "link"],
                   ["id", "movie_id", "company_id", "company_type_id"],
                   ["id", "movie_id", "info_type_id"],
                   ["id", "movie_id", "info_type_id"],
                   ["id", "movie_id", "keyword_id"],
                   ["id", "movie_id", "linked_movie_id", "link_type_id"],
                   ["id"],
                   ["id", "person_id", "info_type_id"],
                   ["id", "role"],
                   ["id", "kind_id"]
                  ]

tableFilterColsList = [["note", "role_id", "nr_order"],
                   [],
                   [],
                   [],
                   [],
                   ["country_code"],
                   ["kind"],
                   [],
                   ["info"],
                   ["keyword"],
                   [],
                   [],
                   ["company_type_id","note"],
                   ["info_type_id"],
                   ["info", "info_type_id"],
                   ["keyword_id"],
                   [],
                   ["gender", "name"],
                   [],
                   ["role"],
                   ["episode_nr", "season_nr", "production_year", "phonetic_code", "series_years", "kind_id"]
                ]

In [9]:
dbConn = getIMDBConn()
for i in range(len(tableNamesList)):
    for filterCol in tableFilterColsList[i]:
        for filterCol2 in tableFilterColsList[i]:
            if filterCol == filterCol2:
                continue    
            dbConn.dropStatistics(filterCol, filterCol2, tableNamesList[i])

In [39]:
# Test 1: Create statistics on role_id + nr_order
dbConn = getIMDBConn()
for i in range(len(tableNamesList)):
    for filterCol in tableFilterColsList[i]:
        for filterCol2 in tableFilterColsList[i]:
            if filterCol == filterCol2:
                continue
            dbConn.createStatistics(filterCol, filterCol2, tableNamesList[i])
dbConn.runAnalyze()